In [3]:
# Импортирование необходимых библиотек
import matplotlib.pyplot as plt   # Отрисовка изображений в Colab
from PIL import Image, ImageDraw, ImageOps  # Библиотека для работы с изображениями
import numpy as np

# Работа с гугл диском
from google.colab import drive
from google.colab import files

# Возможно пригодится
# import numpy as np # Импортируем библиотеку numpy
# import time
# import random
# import math
# import os

In [4]:
# Монтируем наш Google-диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Настройка

dbFile = '/content/drive/MyDrive/Colab Notebooks/Machine_learning/Datasets'  # Файл с исходными изображениями
outDir = '/content/cars_dataset'          # Директория, куда складываем все получившиеся в ходе обработки файлы
db     = '/content/cars'

# Переменная для перевода в вероятности (One Hot Encoding)
labels = {} # Используем словарь
labels['Ferrari']  = [1,0,0]
labels['Mercedes'] = [0,1,0]
labels['Renault']  = [0,0,1]

In [ ]:
# Трансформации

# Поворот изображения в градусах
rotation = [-9, -6, 0, 6, 9]

# Приближене изображени в процентах
zoom = [0, 6, 12]

# Размер финального изображения
final_size = [ 192, 108 ]

In [ ]:
# Удаляем выходную папку и создаем новую
!rm -rf {outDir}
!rm -rf {db}

In [ ]:
# Распаковываем файл
!unzip -q {dbFile} -d {db}

In [ ]:
# Создаем папку, куда будем складывать изображения
os.mkdir(outDir)
# Создаем подпапки
os.mkdir(outDir+"/Ferrari")
os.mkdir(outDir+"/Mercedes")
os.mkdir(outDir+"/Renault")

In [ ]:
# Пробегаемся по всем изображениям и генерим новые

dir_list = os.listdir(db)
for d in dir_list: #Проход по папкам
  cfile = 0
  fid = os.listdir(db+"/"+d)
  for f in fid:
    if '.png' not in f:
      continue

    cfile += 1
    print(cfile, "из", len(fid), "->",f)

    file_car = db+"/"+d+"/"+f # Получаем путь к изображению

    # Сохраняем файл по пути
    img_car = Image.open(file_car)

    width  = img_car.size[0]    # Получаем ширину
    height = img_car.size[1]    # Получаем высоту

                    # Производим трансформации

    # Делаем поворот с наложением на существующую картинку
    img_car_t = img_car.copy()
    for r in rotation:
      if r != 0:
        
        img_car_tb = img_car_t.copy()
        img_car_tb = img_car_tb.convert('RGBA')

        # сохраняем неповернутое изображение как задний фон
        img_background_car = img_car_tb.copy()

        # Поворачиваем изображение
        img_car_tb = img_car_tb.rotate(r)

        img_background_car.paste(img_car_tb, (0,0), img_car_tb)

        # Конвертируем обратно в изображение без альфа-канала
        img_car_tr = img_background_car.convert('RGB')
      else:
        img_car_tr = img_car_t.copy()

    for z in zoom:
      if z != 0:
        newsize = [ int(width + width/100*z) , int(height + height/100*z) ]
        img_car_trz = img_car_tr.resize(newsize)
      else:
        newsize = [ width, height ]
        img_car_trz = img_car_tr.copy()


      p1 = (newsize[0] - width) // 2
      p2 = (newsize[1] - height) // 2
      p3 = p1 + width
      p4 = p2 + height

      img_car_trzc = img_car_trz.crop((p1, p2, p3, p4))

      fname_car = 'z' + str(z) + "_r" + str(r) + "_" + f
      fpath_car = outDir + "/" + d + "/" + fname_car

      img_car_trzc.save(fpath_car, "PNG")


1 из 1180 -> car__74.png
2 из 1180 -> car__879.png
3 из 1180 -> car__301.png
4 из 1180 -> car__684.png
5 из 1180 -> car__800.png
6 из 1180 -> car__645.png
7 из 1180 -> car__83.png
8 из 1180 -> car__425.png
9 из 1180 -> car__1338.png
10 из 1180 -> car__529.png
11 из 1180 -> car__424.png
12 из 1180 -> car__715.png
13 из 1180 -> car__1157.png
14 из 1180 -> car__706.png
15 из 1180 -> car__755.png
16 из 1180 -> car__1250.png
17 из 1180 -> car__316.png
18 из 1180 -> car__265.png
19 из 1180 -> car__1008.png
20 из 1180 -> car__1036.png
21 из 1180 -> car__1070.png
22 из 1180 -> car__1277.png
23 из 1180 -> car__1150.png
24 из 1180 -> car__94.png
25 из 1180 -> car__1053.png
26 из 1180 -> car__822.png
27 из 1180 -> car__1041.png
28 из 1180 -> car__1176.png
29 из 1180 -> car__256.png
30 из 1180 -> car__0.png
31 из 1180 -> car__122.png
32 из 1180 -> car__1.png
33 из 1180 -> car__1210.png
34 из 1180 -> car__543.png
35 из 1180 -> car__1208.png
36 из 1180 -> car__1372.png
37 из 1180 -> car__112.png
38 

In [ ]:
# Знакомая нам функция загрузки изображения
def load_image(file_name):
  image = Image.open(file_name) # Загружаем изображение из файла

  image = image.convert('RGB')  # Приводим изображение к режиму черно-белого изображения 8 бит (0 ... 256)

  image_width  = image.size[0]  # Получаем ширину
  image_height = image.size[1]  # Получаем высоту

  image_data = image.getdata()          # Получаем данные картинки
  image_data = np.array(image_data)     # Преобразуем в np массив

  image = image_data.reshape(image_height, image_width, 3)  # Решейпим изображение для цветного варианта
  if (image_height != 108) | (image_width != 192):
    print(file_name, "!!!!!!!!!!!!!!")

  return image # Возвращаем трехмерный массив с пикселями

In [ ]:
# Формируем наш датасет

image_list = [] #Список путей ко всем изображениям

for label in labels.keys():
  flist = os.listdir(outDir + "/" + label)
  print (label, '->', len(flist), 'изображений')
  for image_name in flist:
    if ".png" not in image_name:
      continue
    image_path = outDir + "/" + label + "/" + image_name
    image_list.append(image_path)

Ferrari -> 3270 изображений
Mercedes -> 3483 изображений
Renault -> 3540 изображений


In [ ]:
print()
print("Пример путей к файлам в массиве:")
print(image_list[0:3])

# перемешаем значения в датасете
random.shuffle(image_list)

print()
print("Пример путей к файлам в массиве после перемешивания:")
print(image_list[0:3])

print()


Пример путей к файлам в массиве:
['/content/cars_dataset/Ferrari/z0_r9_car_Ferrari__656.png', '/content/cars_dataset/Ferrari/z6_r9_car_Ferrari__880.png', '/content/cars_dataset/Ferrari/z0_r9_car_Ferrari__404.png']

Пример путей к файлам в массиве после перемешивания:
['/content/cars_dataset/Renault/z0_r9_car__211.png', '/content/cars_dataset/Ferrari/z0_r9_car_Ferrari__237.png', '/content/cars_dataset/Ferrari/z0_r9_car_Ferrari__12.png']



In [ ]:
x_train = []
y_train = []

c = 1
flen = len(image_list)

for image_name in image_list:
  img = load_image(image_name)
  label = image_name.split('/')[3]
  x_train.append(img)
  y_train.append(labels[label]) # Добавляем верный ответ соответствующий изображению в формате ОНЕ
  print(c, flen, image_name) # Для отслеживания процесса загрузки
  c += 1

x_train = np.array(x_train)
y_train = np.array(y_train)

1 10293 /content/cars_dataset/Renault/z0_r9_car__211.png
2 10293 /content/cars_dataset/Ferrari/z0_r9_car_Ferrari__237.png
3 10293 /content/cars_dataset/Ferrari/z0_r9_car_Ferrari__12.png
4 10293 /content/cars_dataset/Ferrari/z12_r9_car_Ferrari__623.png
5 10293 /content/cars_dataset/Mercedes/z6_r9_car__1131.png
6 10293 /content/cars_dataset/Mercedes/z6_r9_car__270.png
7 10293 /content/cars_dataset/Mercedes/z6_r9_car__776.png
8 10293 /content/cars_dataset/Mercedes/z12_r9_car__288.png
9 10293 /content/cars_dataset/Ferrari/z6_r9_car_Ferrari__1289.png
10 10293 /content/cars_dataset/Mercedes/z12_r9_car__163.png
11 10293 /content/cars_dataset/Ferrari/z0_r9_car_Ferrari__1086.png
12 10293 /content/cars_dataset/Renault/z12_r9_car__62.png
13 10293 /content/cars_dataset/Ferrari/z12_r9_car_Ferrari__1047.png
14 10293 /content/cars_dataset/Mercedes/z12_r9_car__1145.png
15 10293 /content/cars_dataset/Mercedes/z6_r9_car__569.png
16 10293 /content/cars_dataset/Ferrari/z6_r9_car_Ferrari__329.png
17 10293 

In [ ]:
# Выведем размерности нашего датасета
print("Размерность x_train", x_train.shape)
print()
print("Размерность y_train", y_train.shape)

In [ ]:
# Разобъем наш датасет на выборки

# Проверочная выборка
x_val = x_train[9000:10000]
y_val = y_train[9000:10000]

# Тестовая выборка
x_test = x_train[10000:]
y_test = y_train[10000:]

# Обучающая выборка
x_train = x_train[0:9000]
y_train = y_train[0:9000]

print("Обучающая выборка:")
print(x_train.shape)
print(y_train.shape)
print()

print("Проверочная выборка:")
print(x_val.shape)
print(y_val.shape)
print()

print("Тестовая выборка:")
print(x_test.shape)
print(y_test.shape)

In [ ]:
# задаём batch_size
batch_size = 8*6

# Создаем последовательную модель
model = Sequential()
# Слой пакетной нормализации
model.add(BatchNormalization(input_shape=(108, 192, 3), name="bn1"))
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', name="Conv2D-layer1"))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', name="Conv2D-layer2"))
# Первый слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer1"))
# Слой Dropout
model.add(Dropout(0.25, name="Dropout-layer1"))

# Слой пакетной нормализации
model.add(BatchNormalization(name = "bn2"))
# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name = "Conv2D-layer3"))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name = "Conv2D-layer4"))
# Второй слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer2"))
# Слой Dropout
model.add(Dropout(0.25, name="Dropout-layer2"))

# Слой пакетной нормализации
model.add(BatchNormalization(name = "bn3"))
# Пятый сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name = "Conv2D-layer5"))
# Шестой сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name = "Conv2D-layer6"))
# Третий слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer3"))
# Слой регуляризации Dropout
model.add(Dropout(0.25, name="Dropout-layer3"))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten(name="flatten"))
# Выходной полносвязный классифицирующий слой
model.add(Dense(3, activation='softmax', name = "labels"))

# Компилируем сеть
#model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.01), metrics=["accuracy"])
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# Обучаем сеть на наших данных
history = model.fit(x_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=20,
                    validation_data=(x_val, y_val),
                    verbose=1)

# Отображаем график точности обучения
plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()